### In this lab we will implement the circuit to construct a Bell pair using Cirq, and play around with the results.

First we'll take care of the logistics: upgrading pip and installing cirq into our system.

In [1]:
import sys
!pip install --upgrade pip
!{sys.executable} -m pip install cirq
# These might take a bit to run!

     |████████████████████████████████| 1.4MB 6.3MB/s eta 0:00:01
  Found existing installation: pip 19.0.3
    Uninstalling pip-19.0.3:
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/bin/pip'
Consider using the `--user` option or check the permissions.



Now that cirq is in the system, let's import it into our program

In [2]:
import cirq
# This also takes a while to run.

Now that we got everything set up, lets verify that it works by running the cell below. You should get output that looks like this:
```
(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)
```

In [3]:
print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)


If the pattern above didn't show up, let me know. 

Now let's jump into the actual program. First, lets initialize some qubits! When we initialize qubits in Cirq, we have to specify two coordinates for the position of the qubit, or where it is placed in a grid. One can think of the components as (*x, y*) coordinates. (Looking at the foxtail example above, it appears that the components might be flipped from conventional, but I guess that's just kinda the way it is). 

Let's make our 2 qubits, and put them into a list.

In [4]:
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)
qubits = [q0, q1]

print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(1, 0)]


Cool, now let's initialize a circuit!

In [5]:
circuit = cirq.Circuit()

Since we're making a Bell circuit, we want to apply the `H` Gate to `q0` and `CNOT` on both, with `q0` as the control. The way we add gates to a circuit is by `append`ing, like how you would add elements to a Python list.


In [6]:
circuit.append(cirq.H(q0))
circuit.append(cirq.CNOT.on(q0, q1))

Let's print out the circuit and see what it looks like so far

In [7]:
print(circuit)

(0, 0): ───H───@───
               │
(1, 0): ───────X───


As expected, we have `H` on the first qubit and `CNOT` on both. Now let's add some measurement operators!

In [8]:
circuit.append(cirq.measure(*qubits, key='x'))  
# *qubits means to include all the qubits in this measurement, and 'x' just gives this measurement a name.
print(circuit)

(0, 0): ───H───@───M('x')───
               │   │
(1, 0): ───────X───M────────


At this point let's check in and see that we're all on the same page. You should have gotten this output after running the cell above:

```
(0, 0): ───H───@───M('x')───
               │   │
(1, 0): ───────X───M────────
```

If you didn't get this output, you probably ran the cells in a different order than what is listed. Try to go back and re-run the cells starting at the one where we initialized the qubits. If you're still getting a different result after that, go to the Tab that says "Kernel", click "Restart & Clear Output", and run through the cells one by one. You should end up with the correct result then.

Cool, now that we have a circuit, let's go ahead and try to measure it! It would be super tedious to run it manually many many times. Fortunately, Cirq has this handy dandy Simulator that we can use. Let's make one now.

In [9]:
simulator = cirq.Simulator()

In [10]:
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({0: 51, 3: 49})


So we've just simulated the circuit by running it 100 times and printing it out. As we expect, the system is in the bell state, so roughly half the time we'd get the result `0` and half the times we'd get the result `3`. (3 is the state |11>). You can go back and re-run the above cell many times to play with it a little more. Pretty cool huh!

But what if we didn't want to measure q0 and q1 at the same time? Let's say we want to measure q0 first, and then q1. Then we would expect that the measurement of q1 would by exactly the same as whatever we measured in q0, since the 2 qubits are entangled. Let's try it out.

In [11]:
# do the same thing here as what we did before.
circuit = cirq.Circuit()
circuit.append(cirq.H(q0))
circuit.append(cirq.CNOT.on(q0, q1))

In [12]:
# append a measurement operator on q0 only, and name this measurement 'x'
circuit.append(cirq.measure(q0, key='x'))

Now we want to append a measurement operator on q1, but it should occur a *moment* after q0's measurement. The way we do this is to specify an append strategy, `NEW_THEN_INLINE`. This strategy specifies that we insert the gate into the next available moment, instead of the earliest possible moment, which is the default behavior if no strategy is specified. (And that's what happened in the previous example, the measurement on q1 was put immediately after the CNOT). Note that now measuring q0 and q1 are 2 different measurements, we have to give them different names, and I chose 'y' cuz why not.

In [13]:
circuit.append(cirq.measure(q1, key='y'), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
print(circuit)

(0, 0): ───H───@───M('x')────────────
               │
(1, 0): ───────X────────────M('y')───


You should now see that the measurement on q1 occurs a moment after q0. If not, re-run the last 3 cells.

Now let's simulate our circuit and print some results!

In [14]:
simulator = cirq.Simulator()
results = simulator.run(circuit, repetitions=100)

In [15]:
print(f"Measuring first qubit: {results.histogram(key='x')}")
print(f"Measuring second qubit: {results.histogram(key='y')}")

Measuring first qubit: Counter({0: 51, 1: 49})
Measuring second qubit: Counter({0: 51, 1: 49})


Observe how the results of second qubit always matches the first qubit! If you set `repetitions` to 1, you can see it on a case by case basis as well. Pretty dope if I say so myself.

Now that you know the basics of Cirq, go crazy with it! (or don't I'm not ur mom xd)